# Formula 1 data analysis application season 2025
This is the 2025 version

This time, I use fastf1 for race data

<h2 style="color: red;">DON'T FORGET TO UPDATE RACE INDEX</h2>

In [1]:
from racenameindex import RaceNameIndex

race_idx = RaceNameIndex.Australia
race_year = 2024

In [14]:
# BackEnd
import fastf1 as ff1
import fastf1.plotting as ff1plt
import numpy as np
import pandas as pd
import plotly.express as px
from fastf1.core import Laps
from globaldata import update_data

# init
ff1plt.setup_mpl(mpl_timedelta_support=True, misc_mpl_mods=False, color_scheme='fastf1')

# last race data
race = ff1.get_session(race_year, race_idx, 'R')
race.load()
driver_laps = race.laps.pick_drivers(race.drivers)
driver_laps = driver_laps.reset_index()[driver_laps["IsAccurate"] == True] 
driver_laps["LapTime(s)"] = driver_laps["LapTime"].dt.total_seconds()
driver_laps["Time(s)"] = driver_laps["Time"].dt.total_seconds()
average_lap_time = driver_laps.groupby("Driver")["LapTime(s)"].mean().reset_index()
fastest = average_lap_time["LapTime(s)"].min()
average_lap_time['LapTimeDelta'] = average_lap_time['LapTime(s)'] - fastest
finishing_order = [race.get_driver(i)["Abbreviation"] for i in race.drivers]
average_lap_time["Position"] = average_lap_time["Driver"].apply(lambda x: finishing_order.index(x) + 1)
average_lap_time = average_lap_time.sort_values(by="Position")
# average_lap_time["Driver"] = average_lap_time["Driver"].str

# qualifying data
quali_session = ff1.get_session(race_year, race_idx, 'Q')
quali_session.load()
quali_data = quali_session.laps.pick_drivers(quali_session.drivers)
quali_data = quali_data.reset_index()[quali_data["IsAccurate"] == True]
list_fastest_laps = list()
for drv in quali_data["Driver"].unique():
    drvs_fastest_lap = quali_data.pick_driver(drv).pick_fastest()
    list_fastest_laps.append(drvs_fastest_lap)
fastest_laps = Laps(list_fastest_laps) \
    .sort_values(by='LapTime') \
    .reset_index(drop=True)
pole_lap = fastest_laps.pick_fastest()
fastest_laps['LapTimeDelta'] = fastest_laps['LapTime'] - pole_lap['LapTime']
fastest_laps["LapTime"] = fastest_laps["LapTime"].dt.total_seconds()
fastest_laps["LapTimeDelta"] = fastest_laps["LapTimeDelta"].dt.total_seconds()
# fastest_laps["Driver"] = fastest_laps["Driver"].str

# backend for 2 fastest drivers comparison
result = quali_session.results
r1 = result.loc[result['Position'] == 1]
r2 = result.loc[result['Position'] == 2]
lap = quali_session.laps.pick_driver(r1['Abbreviation'][0]).pick_fastest()
lap2 = quali_session.laps.pick_driver(r2['Abbreviation'][0]).pick_fastest()
colormap = {r1['Abbreviation'][0]: ff1plt.get_driver_color(r1['Abbreviation'][0], quali_session), r2['Abbreviation'][0]: ff1plt.get_driver_color(r2['Abbreviation'][0], quali_session)}
tel = lap.get_telemetry()
tel2 = lap2.get_telemetry()
driver = lap.Driver
driver2 = lap2.Driver
x = np.array(tel['X'].values)
y = np.array(tel['Y'].values)
x2 = np.array(tel2['X'].values)
y2 = np.array(tel2['Y'].values)
def cumDist(x, y):
    dist = np.sqrt(np.diff(x)**2 + np.diff(y)**2)
    cumdist = np.cumsum(dist)
    return np.concatenate(([0], cumdist))
cd = cumDist(x, y)
cd2 = cumDist(x2, y2)
time = np.array(tel['Time'].values)
time2 = np.array(tel2['Time'].values)
dif = []
for i in range(len(cd)):
    cpd = cd[i]
    t = time[i]
    if t not in time2:
        cpd2 = np.interp(np.float64(t), np.float64(time2), np.float64(cd2))
    else:
        cpd2 = cd2[np.where(time2 == t)[0][0]]
    if cpd2 > cpd:
        dif.append(driver2)
    else:
        dif.append(driver)
qualy2 = pd.DataFrame({'X': x, 'Y': y, 'Driver': dif})

# gap to winner
winner_lap = driver_laps[driver_laps["Position"] == 1].iloc[-1]
winner_driver = winner_lap["Driver"]
winner_times = driver_laps[driver_laps["Driver"] == winner_driver][["LapNumber", "Time(s)"]].set_index("LapNumber")
driver_laps = driver_laps.merge(winner_times, on="LapNumber", suffixes=("", "_Winner"))
driver_laps['GapToWinner'] = driver_laps['Time(s)'] - driver_laps['Time(s)_Winner']

# stints
stints = driver_laps[["Driver", "Stint", "Compound", "LapNumber"]]
stints = stints.groupby(["Driver", "Stint", "Compound"])
stints = stints.count().reset_index()
stints = stints.rename(columns={"LapNumber": "StintLength"})
stint_idxs = {drv: [1] for drv in stints["Driver"].unique()}
# stints["Driver"] = stints["Driver"].str

# colors
colors = ff1plt.get_driver_color_mapping(session=race)
compound_colors = {}
for compound in stints["Compound"].unique():
    compound_colors[compound] = ff1plt.get_compound_color(compound, session=race)

# global year
global_df = update_data(race_year, race_idx)

core           INFO 	Loading data for Bahrain Grand Prix - Race [v3.4.0]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['1', '11', '55', '16', '63', '4', '44', '81', '14', '18', '24', '20', '3', '22', '23', '27', '31', '10', '77', '2']
core           INFO 	Loading data for Bahrain Grand Prix - Qua

No new data


In [17]:
# get the average lap time per driver
px.scatter(average_lap_time, x="Driver", y="LapTime(s)", title="Average lap time per driver", template="plotly_dark")

In [6]:
# line plot of the lap times per driver per lap with color per driver
px.line(driver_laps, x="LapNumber", y="LapTime(s)", color="Driver", title="Lap times per driver per lap", template="plotly_dark", color_discrete_map=colors)

In [7]:
# position evolution per driver per lap (reverse y-axis)
fig = px.line(driver_laps, x="LapNumber", y="Position", color="Driver", title="Position evolution per driver", template="plotly_dark", color_discrete_map=colors)
fig.update_yaxes(autorange="reversed")
fig

In [16]:
# type of tyre per driver per lap (horizontal bar plot)
fig = px.bar(stints, x="StintLength", y="Driver", color="Compound", title="Stint length per driver", template="plotly_dark", color_discrete_map=compound_colors, orientation="h")
fig.update_yaxes(tickmode="linear")
fig

In [17]:
# gap to winner
fig = px.line(driver_laps, x="LapNumber", y="GapToWinner", color="Driver",
              title="Écart temporel au vainqueur par pilote",
              template="plotly_dark", color_discrete_map=colors)
fig.update_yaxes(autorange="reversed")
fig

In [15]:
# qualifying time
fig = px.scatter(fastest_laps, x="Driver", y="LapTime", title="Qualifying lap time delta to pole", template="plotly_dark", color="Driver", color_discrete_map=colors)

# lines constants
hundredsevenrule = pole_lap["LapTime"] * 1.07
hundredsevenrule = hundredsevenrule.total_seconds()
q1toq2 = 15
q2toq3 = 10

# add lines
fig.add_hline(y=hundredsevenrule, line_dash="dot", line_color="red", annotation_text="107% rule", annotation_position="bottom right") # 107% rule
fig.add_vline(x=q1toq2, line_dash="dot", line_color="red", annotation_text="eliminated in Q1", annotation_position="bottom right") # Q1 to Q2
fig.add_vline(x=q2toq3, line_dash="dot", line_color="red", annotation_text="eliminated in Q2", annotation_position="bottom right") # Q2 to Q3

In [9]:
# 2 fastest drivers of qualy comparison
px.scatter(qualy2, x='X', y='Y', color='Driver', color_discrete_map=colormap, title=f"{r1["Abbreviation"][0]} vs {r2["Abbreviation"][0]} in qualy", template="plotly_dark")

C:\Users\Shayane\AppData\Local\Temp\ipykernel_14500\966582036.py:2: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

